In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir3='/Volumes/My Passport/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
index=646
ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(2000)+'.nc')
ar0=ds['ar_labeled'][index,:,:]

ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(2000)+'.nc')
ivt=ds['ivt'][index*2,:,:]

fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)
ax.pcolormesh(ar0.lon.values,ar0.lat.values,ar0.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')
    
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{variable}$',fontsize=45,pad=5)
plt.show()
fig.savefig(dir_data+'ar_demo_1.png')

#######

ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(2000)+'.nc')
ar=ds['ar_labeled'][index*2,:,:]

values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
max_used=ar0.max().values.tolist()
for value in values:
    select_component=ar.where(ar==value)
    overlap=ar0.where(select_component>0)
    values_original=np.unique(overlap.values)
    values_original=[x for x in values_original if x>0]
    nums_list=[]
    for i in range(len(values_original)):
        sel_1=overlap.where(overlap==values_original[i])
        number=len([x for x in sel_1.values.flatten().tolist() if x>0])
        nums_list.append(number)
    zipped=list(zip(values_original,nums_list))
    ordered=sorted(zipped,key=lambda x:x[1])
    max_num=ordered[-1][0]
    
    if max_num in np.unique(new_field.values):
        max_num=max_used+1
        max_used=max_used+1
    
    select_component=select_component/select_component*max_num
    select_component=select_component.fillna(0)
    
    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)

fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')
    
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{fixed}$',fontsize=45,pad=5)
plt.show()
fig.savefig(dir_data+'ar_demo_2.png')

ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(2000)+'.nc')
ar=ds['ar_labeled'][index,:,:]

values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
max_used=ar0.max().values.tolist()
for value in values:
    select_component=ar.where(ar==value)
    overlap=ar0.where(select_component>0)
    values_original=np.unique(overlap.values)
    values_original=[x for x in values_original if x>0]
    nums_list=[]
    for i in range(len(values_original)):
        sel_1=overlap.where(overlap==values_original[i])
        number=len([x for x in sel_1.values.flatten().tolist() if x>0])
        nums_list.append(number)
    zipped=list(zip(values_original,nums_list))
    ordered=sorted(zipped,key=lambda x:x[1])
    max_num=ordered[-1][0]
    
    if max_num in np.unique(new_field.values):
        max_num=max_used+1
        max_used=max_used+1
    
    select_component=select_component/select_component*max_num
    select_component=select_component.fillna(0)
    
    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)

fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')
    
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=45,pad=5)
plt.show()
fig.savefig(dir_data+'ar_demo_3.png')

########

In [ ]:
index=646

ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(2000)+'.nc')
ar=ds['ar_labeled'][index,:,:]

values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
max_used=ar0.max().values.tolist()
for value in values:
    select_component=ar.where(ar==value)
    overlap=ar0.where(select_component>0)
    values_original=np.unique(overlap.values)
    values_original=[x for x in values_original if x>0]
    nums_list=[]
    for i in range(len(values_original)):
        sel_1=overlap.where(overlap==values_original[i])
        number=len([x for x in sel_1.values.flatten().tolist() if x>0])
        nums_list.append(number)
    zipped=list(zip(values_original,nums_list))
    ordered=sorted(zipped,key=lambda x:x[1])
    max_num=ordered[-1][0]
    
    if max_num in np.unique(new_field.values):
        max_num=max_used+1
        max_used=max_used+1
    
    select_component=select_component/select_component*max_num
    select_component=select_component.fillna(0)
    
    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)


ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(2000)+'.nc')
ivt=ds['ivt'][index,:,:]

from matplotlib.colors import LinearSegmentedColormap
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors=['w','b'])

fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')
    
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{variable}$',fontsize=45,pad=5)

    
plt.show()
fig.savefig(dir_data+'ar_demo_4.png')


ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(2000)+'.nc')
ar=ds['ar_labeled'][index,:,:]

values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
max_used=ar0.max().values.tolist()
for value in values:
    select_component=ar.where(ar==value)
    overlap=ar0.where(select_component>0)
    values_original=np.unique(overlap.values)
    values_original=[x for x in values_original if x>0]
    nums_list=[]
    for i in range(len(values_original)):
        sel_1=overlap.where(overlap==values_original[i])
        number=len([x for x in sel_1.values.flatten().tolist() if x>0])
        nums_list.append(number)
    zipped=list(zip(values_original,nums_list))
    ordered=sorted(zipped,key=lambda x:x[1])
    max_num=ordered[-1][0]
    
    if max_num in np.unique(new_field.values):
        max_num=max_used+1
        max_used=max_used+1
    
    select_component=select_component/select_component*max_num
    select_component=select_component.fillna(0)
    
    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)


fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{fixed}$',fontsize=45,pad=5)
    
plt.show()
fig.savefig(dir_data+'ar_demo_5.png')

ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(2000)+'.nc')
ar=ds['ar_labeled'][index,:,:]

values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
max_used=ar0.max().values.tolist()
for value in values:
    select_component=ar.where(ar==value)
    overlap=ar0.where(select_component>0)
    values_original=np.unique(overlap.values)
    values_original=[x for x in values_original if x>0]
    nums_list=[]
    for i in range(len(values_original)):
        sel_1=overlap.where(overlap==values_original[i])
        number=len([x for x in sel_1.values.flatten().tolist() if x>0])
        nums_list.append(number)
    zipped=list(zip(values_original,nums_list))
    ordered=sorted(zipped,key=lambda x:x[1])
    max_num=ordered[-1][0]
    
    if max_num in np.unique(new_field.values):
        max_num=max_used+1
        max_used=max_used+1
    
    select_component=select_component/select_component*max_num
    select_component=select_component.fillna(0)
    
    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)


fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.where(ar>0),vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=45,pad=5)
    
plt.show()
fig.savefig(dir_data+'ar_demo_6.png')

In [ ]:
index=646

ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(2000)+'.nc')
ar=ds['ar_labeled'][index,:,:]
ar0=ar0.interp_like(ar)

values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
max_used=ar0.max().values.tolist()
for value in values:
    select_component=ar.where(ar==value)
    overlap=ar0.where(select_component>0)
    values_original=np.unique(overlap.values)
    values_original=[x for x in values_original if x>0]
    nums_list=[]
    for i in range(len(values_original)):
        sel_1=overlap.where(overlap==values_original[i])
        number=len([x for x in sel_1.values.flatten().tolist() if x>0])
        nums_list.append(number)
    zipped=list(zip(values_original,nums_list))
    ordered=sorted(zipped,key=lambda x:x[1])
    max_num=ordered[-1][0]
    
    if max_num in np.unique(new_field.values):
        max_num=max_used+1
        max_used=max_used+1
    
    select_component=select_component/select_component*max_num
    select_component=select_component.fillna(0)
    
    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)


ds=xr.open_dataset(dir3+'jra55_ivt_'+str(2000)+'.nc')
ivt=ds['ivt'][index,:,:]

from matplotlib.colors import LinearSegmentedColormap
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors=['w','b'])

fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')
    
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{variable}$',fontsize=45,pad=5)

    
plt.show()
fig.savefig(dir_data+'ar_demo_7.png')


ds=xr.open_dataset(dir_data+'jra55.ar.labels.model.'+str(2000)+'.nc')
ar=ds['ar_labeled'][index,:,:]

values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
max_used=ar0.max().values.tolist()
for value in values:
    select_component=ar.where(ar==value)
    overlap=ar0.where(select_component>0)
    values_original=np.unique(overlap.values)
    values_original=[x for x in values_original if x>0]
    nums_list=[]
    for i in range(len(values_original)):
        sel_1=overlap.where(overlap==values_original[i])
        number=len([x for x in sel_1.values.flatten().tolist() if x>0])
        nums_list.append(number)
    zipped=list(zip(values_original,nums_list))
    ordered=sorted(zipped,key=lambda x:x[1])
    max_num=ordered[-1][0]
    
    if max_num in np.unique(new_field.values):
        max_num=max_used+1
        max_used=max_used+1
    
    select_component=select_component/select_component*max_num
    select_component=select_component.fillna(0)
    
    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)


fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{fixed}$',fontsize=45,pad=5)
    
plt.show()
fig.savefig(dir_data+'ar_demo_8.png')

ds=xr.open_dataset(dir2+'jra55.ar.labels.model.poleward.'+str(2000)+'.nc')
ar=ds['ar_labeled'][index,:,:]

values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
max_used=ar0.max().values.tolist()
for value in values:
    select_component=ar.where(ar==value)
    overlap=ar0.where(select_component>0)
    values_original=np.unique(overlap.values)
    values_original=[x for x in values_original if x>0]
    nums_list=[]
    for i in range(len(values_original)):
        sel_1=overlap.where(overlap==values_original[i])
        number=len([x for x in sel_1.values.flatten().tolist() if x>0])
        nums_list.append(number)
    zipped=list(zip(values_original,nums_list))
    ordered=sorted(zipped,key=lambda x:x[1])
    max_num=ordered[-1][0]
    
    if max_num in np.unique(new_field.values):
        max_num=max_used+1
        max_used=max_used+1
    
    select_component=select_component/select_component*max_num
    select_component=select_component.fillna(0)
    
    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)


fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.where(ar>0),vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=45,pad=5)
    
plt.show()
fig.savefig(dir_data+'ar_demo_9.png')

In [ ]:
index=646

ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(2000)+'.nc')
ar=ds['ar_labeled'][index,:,:]
ar0=ar0.interp_like(ar)

values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
max_used=ar0.max().values.tolist()
for value in values:
    select_component=ar.where(ar==value)
    overlap=ar0.where(select_component>0)
    values_original=np.unique(overlap.values)
    values_original=[x for x in values_original if x>0]
    nums_list=[]
    for i in range(len(values_original)):
        sel_1=overlap.where(overlap==values_original[i])
        number=len([x for x in sel_1.values.flatten().tolist() if x>0])
        nums_list.append(number)
    zipped=list(zip(values_original,nums_list))
    ordered=sorted(zipped,key=lambda x:x[1])
    max_num=ordered[-1][0]
    
    if max_num in np.unique(new_field.values):
        max_num=max_used+1
        max_used=max_used+1
    
    select_component=select_component/select_component*max_num
    select_component=select_component.fillna(0)
    
    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)


ds=xr.open_dataset(dir3+'jra55_ivt_'+str(2000)+'.nc')
ivt=ds['ivt'][index,:,:]

from matplotlib.colors import LinearSegmentedColormap
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors=['w','b'])

fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')
    
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{variable}$',fontsize=45,pad=5)

    
plt.show()
fig.savefig(dir_data+'ar_demo_7.png')


In [ ]:
values=np.unique(ar.values)
values=[x for x in values if x>0]
new_field=xr.zeros_like(ar)
for value in values:
    select_component=ar.where(ar==value)
    if value==16:
        select_component=select_component/select_component*22
        select_component=select_component.fillna(0)
    if value==22:
        select_component=select_component/select_component*16
        select_component=select_component.fillna(0)
    else:
        select_component=select_component.fillna(0)

    new_field=new_field+select_component
    
ar=new_field.where(new_field>0)

fig=plt.figure(figsize=(18,12))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)
ax.pcolormesh(ar.lon.values,ar.lat.values,ar.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
ax.coastlines(resolution='10m')
    
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':30,'color':'k'}
g1.ylabel_style={'size':30,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{variable}$',fontsize=45,pad=5)

    
plt.show()
fig.savefig(dir_data+'ar_demo_7.png')


In [ ]:
#Search for anomalies

ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(2000)+'.nc')
ds1=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(2000)+'.nc')
for index in range(687,688):
    print(index)
    ar0=ds['ar_labeled'][index,:,:]
    ivt=ds1['ivt'][index*2,:,:]

    fig=plt.figure(figsize=(18,12))
    ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

    c=ax.contour(ivt.lon,ivt.lat,ivt,levels=[300],colors='k',linewidths=3.0)
    ax.pcolormesh(ar0.lon.values,ar0.lat.values,ar0.values,vmin=0,vmax=ar0.max().values+1,cmap=plt.cm.viridis,alpha=1)
    ax.coastlines(resolution='10m')

    g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
    ax.coastlines(resolution='10m')
    countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
    ax.add_feature(countries)
    g1.xlabel_style={'size':30,'color':'k'}
    g1.ylabel_style={'size':30,'color':'k'}
    g1.xformatter=LONGITUDE_FORMATTER
    g1.yformatter=LATITUDE_FORMATTER
    g1.top_labels=False
    g1.right_labels=False
    g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
    ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{variable}$',fontsize=45,pad=5)
    plt.show()

In [ ]:
fig.savefig(dir_data+'weird_ars_2.png')

In [ ]:
657 - long AR!
687 - case 698
#Look at polar moisture amplifications